In [1]:
using Revise

In [2]:
using MeshCatMechanisms
using MeshCat
using RigidBodyDynamics
using RigidBodySim
using Flux
using JLD2
using Plots; gr()
using LCPSim
using LearningMPC
using LearningMPC.Models
using DataFrames
using Blink

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/Gurobi.ji for module Gurobi.
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/LearningMPC.ji for module LearningMPC.


In [3]:
robot = BoxAtlas(add_contacts=true)
position_bounds(findjoint(mechanism(robot), "floating_base")) .= RigidBodyDynamics.Bounds(-Inf, Inf)
mvis = MechanismVisualizer(robot)
open(mvis, Window())

Blink.AtomShell.Window(1, Blink.AtomShell.Electron(Process(`/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/deps/atom/electron /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/src/AtomShell/main.js port 4805`, ProcessRunning), TCPSocket(RawFD(54) active, 0 bytes waiting), Dict{String,Any}(Pair{String,Any}("callback", Blink.#1))), Blink.Page(1, WebSockets.WebSocket{TCPSocket}(TCPSocket(RawFD(56) active, 0 bytes waiting), true, CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#111),Pair{String,Any}("callback", Blink.#1)), Future(1, 1, 1, Nullable{Any}(true))))

In [67]:
@load "boxatlas-regularized.jld2" datasets net lqrsol mpc_params
mpc_params_data = mpc_params
mpc_params = MPCParams(robot)
mpc_params.Δt = mpc_params_data["Δt"]
mpc_params.horizon = mpc_params_data["horizon"]
net_cost = LearningMPC.LearnedCost(lqrsol, net)
net_mpc_params = MPCParams(robot)
net_mpc_params.horizon = 1
net_mpc_controller = MPCController(robot, net_mpc_params, net_cost, [lqrsol]);
full_mpc_controller = MPCController(robot, mpc_params, lqrsol, [lqrsol, net_mpc_controller]);
lqr_mpc_controller = MPCController(robot, (p = MPCParams(robot); p.horizon=1; p), lqrsol, [lqrsol])

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only


(::MPCController) (generic function with 1 method)

In [73]:
mpc = LearningMPC.mpc_controller(robot, net_cost, Δt=0.05);

Academic license - for non-commercial use only


In [74]:
state = nominal_state(robot)
set_velocity!(state, findjoint(mechanism(robot), "floating_base"), [-0.5, 0.0, 0.0])
problem = LearningMPC.simulation_problem(state, mpc, 0.01, 2.0);
sol = solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6)
setanimation!(mvis, sol)

true

In [ ]:
table = LearningMPC.run_evaluations(
    lqr_mpc_controller,
    "lqr_mpc",
    robot,
    lqrsol,
    [(1, [0])],
    [(1, linspace(-5, 5, 11)), (3, linspace(-10, 10, 11))];
)

append!(table, 
    LearningMPC.run_evaluations(
    net_mpc_controller,
    "net_mpc",
    robot,
    lqrsol,
    [(1, [0])],
    [(1, linspace(-5, 5, 11)), (3, linspace(-10, 10, 11))];
)
)

table

In [ ]:
jldopen("evaluation_table.jld2", "w") do file
    file["evaluations"] = table
end

In [ ]:
using JLD2
using DataFrames
using Plots; gr()

In [ ]:
@load "evaluation_table.jld2" evaluations
table = evaluations;

In [ ]:
plt = plot(legend=false)
for row in eachrow(table)
    if row[:controller] == "net_mpc"
        success = -π/4 <= row[:qf][3] <= π/4
        if success
            scatter!(plt, [row[:v0][1]], [row[:v0][3]], color = "green", marker = :dot, markersize = 10)
        else
            scatter!(plt, [row[:v0][1]], [row[:v0][3]], color = "red", marker = :x)
        end
    end
end
title!(plt, "Learned Cost-to-go")
xlabel!(plt, "Initial x velocity")
ylabel!(plt, "Initial rotational velocity")
savefig(plt, "learned-cost-evaluation.svg")
plt

In [ ]:
plt = plot(legend=false)
for row in eachrow(table)
    if row[:controller] == "lqr_mpc"
        success = -π/4 <= row[:qf][3] <= π/4
        if success
            scatter!(plt, [row[:v0][1]], [row[:v0][3]], color = "green", marker = :dot, markersize = 10)
        else
            scatter!(plt, [row[:v0][1]], [row[:v0][3]], color = "red", marker = :x)
        end
    end
end
title!(plt, "LQR Cost-to-go")
xlabel!(plt, "Initial x velocity")
ylabel!(plt, "Initial rotational velocity")
savefig(plt, "lqr-cost-evaluation.svg")
plt